In [1]:
def readGenome(filename):
    fullGenome = ""
    with open(filename, 'r') as fh:
        for line in fh:
            if not (line[0] == '>'):
                fullGenome += line.rstrip()
    return fullGenome

In [2]:
phix = readGenome("Wk1-Data/phix.fa")

In [3]:
def naiveMatch(p, t):
    occurances = []
    for i in range(len(t) - len(p) + 1):
        match = True
        for j in range(len(p)):
            if t[i + j] != p[j]:
                match = False
                break
        if match:
            occurances.append(i)
    return occurances

pattern = "ACCT"
text = "ACCTCATTGGCGATACCTACCGGCACCT"
print(naiveMatch(pattern, text))


[0, 14, 24]


In [4]:
import random
def generateReads(genome, readAmount, readLen):
    reads = []
    for _ in range(readAmount):
        # randint generate integers in the closed interval
        start = random.randint(0, len(genome) - readLen)
        reads.append(genome[start : start+readLen])
    return reads

print(generateReads(pattern, 10, 2))

['AC', 'CC', 'CT', 'CC', 'AC', 'CC', 'CC', 'AC', 'CT', 'AC']


In [5]:
reads = generateReads(phix, 100, 100)
numMatched = 0
for r in reads:
    o = naiveMatch(r, phix)
    if (len(o) > 0):
        numMatched += 1
print("%d / %d reads matched" % (numMatched, len(reads)))

100 / 100 reads matched


In [6]:
def reverseComplement(s):
    complement = {'A':'T', 'C':'G', 'T':'A', 'G':'C', 'N':'N'}
    ans = ''
    for base in s:
        ans = complement[base] + ans
    return ans

In [7]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()
            seq = fh.readline().rstrip()
            fh.readline()
            qua = fh.readline().rstrip()
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qua)
    return sequences, qualities

In [8]:
reads, _ = readFastq("Wk1-Data/ERR266411_1.first1000.fastq")

numMatched = 0
for r in reads:
    r = r[:30]
    o = naiveMatch(r, phix)
    o.extend(naiveMatch(reverseComplement(r), phix))
    if (len(o) > 0):
        numMatched += 1
print("%d / %d reads matched" % (numMatched, len(reads)))

932 / 1000 reads matched


In [12]:
def naiveWithRC(p, t):
    occurances = []
    rc = reverseComplement(p)
    for i in range(len(t) - len(p) + 1):
        match = True
        for j in range(len(p)):
            if t[i + j] != p[j]:
                match = False
                break
        if match:
            occurances.append(i)
        else:
            match = True
            for j in range(len(rc)):
                if t[i + j] != rc[j]:
                    match = False
                    break
            if match:
                occurances.append(i)
    return occurances

In [13]:
p = 'CCC'
ten_as = 'AAAAAAAAAA'
t = ten_as + 'CCC' + ten_as + 'GGG' + ten_as
occurrences = naiveWithRC(p, t)
print(occurrences)

[10, 23]


In [14]:
p = 'CGCG'
t = ten_as + 'CGCG' + ten_as + 'CGCG' + ten_as
occurrences = naiveWithRC(p, t)
print(occurrences)

[10, 24]


306
156
150


In [27]:
print(len(naiveWithRC("TTAA", lambdaVirus)))
print(len(naiveMatch("TTAA", lambdaVirus)))

195
195


In [31]:
occurrences = naiveWithRC("ACTAAGT", lambdaVirus)
print(occurrences[0])
occurrences = naiveWithRC("AGTCGA", lambdaVirus)
print(occurrences[0])

26028
450
